# Instructions
Choose anyone of the evaluater's name and comment other evaluater's name
- Assign the response file for all the models accordingly

In [ ]:
def get_model_responses(index, model):
    
    file = None
    if model == "256M-base":
        file = "results/base_256M_eval_results.json"
    elif model == "256M-base+step":
        file = "results/base_256M_eval_think_results.json"
    elif model == "500M-base":
        file = "results/base_500M_eval_results.json"
    elif model == "500M-base+step":
        file = "results/base_500M_eval_think_results.json"
    elif model == "256M-3k":
        file = "results/sft_256M_3k_eval_results.json"
    elif model == "256M-3k+step":
        file = "results/sft_256M_3k_eval_think_results.json"
    elif model == "256M-7k":
        file = "results/sft_256M_7k_eval_results.json"
    elif model == "256M-7k+step":
        file = "results/sft_256M_7k_eval_think_results.json"
    elif model == "500M-3k":
        file = "results/sft_500M_3k_eval_results.json"
    elif model == "500M-3k+step":
        file = "results/sft_500M_3k_think_eval_results.json"
    elif model == "500M-7k":
        file = "results/sft_500M_7k_eval_results.json"
    elif model == "500M-7k+step":
        file = "results/sft_500M_7k_eval_think_results.json"
    else:
        raise Exception("Model not found")
    
    with open(file, 'r') as f:
        data = json.load(f)
    
    return data[index]["output"]

In [ ]:
img_folder_path = '../data/filtered_images_5k'  # ye path set karna hoga

In [ ]:
models = ['256M-base', '256M-base+step', '256M-3k', '256M-3k+step', 
          '256M-7k', '256M-7k+step',
          '500M-base', '500M-base+step', '500M-3k', '500M-3k+step', 
          '500M-7k', '500M-7k+step']
# all combinations of models
configurations = []
num_models = len(models)
for i in range(num_models):
    for j in range(i + 1, num_models):
        configurations.append((models[i], models[j]))

print(len(configurations))
# evaluater_name = "Divyansh"
# config = configurations[0:20]

evaluater_name = "Sandy"
config = configurations[20:40]

# evaluater_name = "Aniket"
# config = configurations[40:50]

# evaluater_name = "Apoorva"
# config = configurations[50:58]

# evaluater_name = "Anuj"
# config = configurations[58:66]

print(len(config))
config

In [ ]:
import json
from PIL import Image
from matplotlib import pyplot as plt
from IPython.display import clear_output
import time
import random
queries_path = 'test_set_answers.json'
responses_path = './'
with open(queries_path, 'r') as f:
  queries = json.load(f)
print(queries[0])

In [ ]:
def show_and_get_choice(img, query, answer, responseA, responseB):
    clear_output(wait=True)
    # Display image
    plt.imshow(img)
    plt.axis('off')
    plt.title("Image for the Query")

    # Display query and responses
    print(f"\nQuery: {query}")
    print(f"True ans for reference: {answer}")
    print(f"\nModel A Response:\n{responseA}")
    print(f"\nModel B Response:\n{responseB}")
    plt.show()

    # time.sleep(0.1)
    # Create buttons
    resp = int(input("Better? (0: Tie / 1: Model A / 2: Model B/ -1: pause)\n").strip())

    return resp

f = open(f"{responses_path}/responses_{evaluater_name}.json", 'a')

used_indices = set()
# multiple session mein ye overwrite nhi hona chahiye

In [ ]:
import random

In [ ]:

# ek baar mein 20 session lga rha hun
count = 0
while count < 20:
    index = random.randint(0, len(queries) - 1)
    while index < len(queries) and index in used_indices:
        index = random.randint(0, len(queries) - 1)

    used_indices.add(index)
    count += 1
    for c in configurations:
        clear_output(wait=True)
        modelA, modelB = c

        if random.random() < 0.5:
            modelA, modelB = modelB, modelA  # Swap
        responseA = get_model_responses(index, modelA)
        responseB = get_model_responses(index, modelB)
        query = queries[index]
        answer = queries[index]["label"]
        img = Image.open(f"{img_folder_path}/{query['imgname']}")
        choice = show_and_get_choice(img, query['query'], answer, responseA, responseB)
        if choice == -1:
            index = len(queries)
            # exit(0)
            count = 20
            break
        eval_result = {
            'index': index,
            'model1': modelA,
            'model2': modelB,
            'response1': responseA,
            'response2': responseB,
            'choice': choice
        }
        f.write(json.dumps(eval_result) + "\n")
        f.flush()

        index += 1

## Instructions
Put the `response_files` accordingly

In [ ]:
response_files = [
    'votes/responses_Divyansh.json', 
    'votes/responses_Aniket.json', 
    'votes/responses_Anuj.json', 
    'votes/responses_Apoorva.json',
    'votes/responses_Sandy.json',
    ]

all_comparisons = []
for fname in response_files:
    with open(f"{responses_path}/{fname}", 'r') as f:
        for line in f:
            eval_result = json.loads(line)
            all_comparisons.append(eval_result)

# Elo rating calculation
def calculate_elo(rating1, rating2, result, k=32):
    """Calculate Elo ratings after a game result"""
    expected_score_p1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
    expected_score_p2 = 1 / (1 + 10 ** ((rating1 - rating2) / 400))

    # Result values: 1 = player 1 wins, 0 = player 2 wins, 0.5 = draw
    if result == "p1 wins":
        actual_score_p1, actual_score_p2 = 1, 0
    elif result == "p2 wins":
        actual_score_p1, actual_score_p2 = 0, 1
    else:
        actual_score_p1, actual_score_p2 = 0.5, 0.5

    # Calculate new ratings
    new_rating_p1 = rating1 + k * (actual_score_p1 - expected_score_p1)
    new_rating_p2 = rating2 + k * (actual_score_p2 - expected_score_p2)

    return new_rating_p1, new_rating_p2

# Function to calculate Elo ratings for all players
def calculate_elo_ratings(game_results, initial_rating=1500, k=32):
    player_ratings = {}

    for game in game_results:
        p1, p2, result = game

        # Set initial ratings if players haven't played before
        if p1 not in player_ratings:
            player_ratings[p1] = initial_rating
        if p2 not in player_ratings:
            player_ratings[p2] = initial_rating

        # Get current ratings
        rating_p1 = player_ratings[p1]
        rating_p2 = player_ratings[p2]

        # Calculate new ratings
        new_rating_p1, new_rating_p2 = calculate_elo(rating_p1, rating_p2, result, k)

        # Update player ratings
        player_ratings[p1] = new_rating_p1
        player_ratings[p2] = new_rating_p2

    return player_ratings

# Example usage
def get_result(choice):
    if choice == 0:
        return "tie"
    elif choice == 1:
        return "p1 wins"
    else:
        return "p2 wins"
game_results = [(c['model1'], c['model2'], get_result(c['choice'])) for c in all_comparisons]
print(len(game_results))
elo_ratings = calculate_elo_ratings(game_results)
for player, rating in elo_ratings.items():
    print(f"{player}: {rating:.2f}")


In [ ]:
sorted_ratings = sorted(elo_ratings.items(), key=lambda item: item[1])

# Print the sorted items
for key, value in sorted_ratings:
    print(f"{key}: {value}")